# Projeto 1 - Ciência dos Dados

Nome: Camila Soares Magni

Nome: Lincoln Rodrigo Pereira Melo

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [39]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [40]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\camil\OneDrive\Área de Trabalho\INSPER\2° sem\C dados\Projeto1- Cdados\Proj1Cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [41]:
filename = 'airfryer.xlsx'

In [42]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,o twitter vai me fazer comprar uma airfryer,1
1,"quero assar pão de queijo, mas o forno daqui d...",1
2,"bom, aparentemente deu ruim e saiu toda a tint...",1
3,de airfryer a panelas: polishop celebra 22 ano...,0
4,@danielsmarconn mano eu tô louca nessa airfrye...,1


In [43]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação
0,"o bom da airfryer fica tudo pronto rápido, não...",1
1,um app pra calcular se é mais barato fazer a c...,0
2,@raytotsukishiro aumentou o preço de tudo. aqu...,0
3,meti um pão de queijo na airfryer 😌,1
4,"@alcysio @safbf pô alcysio, me adaptei total p...",0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Escolhemos a fritadeira sem óleo airfryer como nosso produto. Para a classificação dos tweets, consideramos todas as mensagens que expressam alguma opinião sobre o produto, sejam elogios ou críticas, como relevantes, assim como as que falam sobre um desejo de adquirir uma airfryer. O resto foi classificado como irrelevante, incluindo receitas e mensagens sobre outros assuntos que não fazem sentido para a análise do produto. 

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [44]:
import pandas as pd
import numpy as np
from IPython.display import display
import re 
import nltk

# Função de limpeza dos textos
def cleanup(text):
    
    punctuation = '[""''!-.:?;“”~1234567890]'                                      # sinais que queremos retirar do texto
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)                              # retira os sinais
    text_subbed = re.sub(r"http\S+", "", text_subbed)                    # retira os URL
    text_subbed = re.sub('@[^\s]+','',text_subbed,flags=re.MULTILINE)    # retira os nomes de usuário
    text_subbed = re.sub(r"kk\S+", "kk", text_subbed)                    # transformar as strings que começam com kk em 'kk'
    text_subbed = re.sub(r"compr\S+", "compr", text_subbed) 
    text_subbed = re.sub(r"limp\S+", "limp", text_subbed) 
    text_subbed = re.sub(r"batat\S+", "batata", text_subbed) 
    aux = nltk.TweetTokenizer()
    text_subbed = aux.tokenize(text_subbed)                              # separa os emojis
    string_final = ' '.join(text_subbed)                                 # string limpa
    return string_final

In [45]:
# Função para tirar as 'Stopwords' em português:
# nltk.download('stopwords') - INSTALAR, caso vc não tenha instalado
def retirar_stopwords(lista):
    lista_limpa = []
    stopwords = nltk.corpus.stopwords.words('portuguese')
    for palavra in lista:
        if palavra not in stopwords:
            lista_limpa.append(palavra)
    return lista_limpa

In [46]:
lista = []

for linha in train['Treinamento']:
    limpo = cleanup(linha.lower())             # Limpa a planilha Treinamento e tira as letras maiúsculas 
    a = limpo.split()
    b = retirar_stopwords(a)
    limpo = ' '.join(b)  
    lista.append(limpo)
    
serie_train = pd.Series(lista)                 # Transforma a coluna da planilha Treinamento em Series
serie_train

0                       twitter vai fazer compr airfryer
1      quero assar pão queijo forno daqui casa bosta ...
2      bom aparentemente deu ruim saiu toda tinta / t...
3      airfryer panelas polishop celebra anos descont...
4                mano tô louca nessa airfryer serase boa
                             ...                        
295                        toma chifre michelle airfryer
296             impedimento airfryer viva philips walita
297     kk literalmente airfryer carga v potência máxima
298                                  airfryer tudo serio
299                   ranço batata palito feita airfryer
Length: 300, dtype: object

In [47]:
train['Treinamento'] = serie_train        # substitui os tweets originais da planilha, pelos tweets limpos
train

,Treinamento,Classificação
0,twitter vai fazer compr airfryer,1
1,quero assar pão queijo forno daqui casa bosta ...,1
2,bom aparentemente deu ruim saiu toda tinta / t...,1
3,airfryer panelas polishop celebra anos descont...,0
4,mano tô louca nessa airfryer serase boa,1
...,...,...
295,toma chifre michelle airfryer,0
296,impedimento airfryer viva philips walita,0
297,kk literalmente airfryer carga v potência máxima,0
298,airfryer tudo serio,1


In [48]:
filtro = train.Classificação==1          # filtra os tweets classificados como relevantes
relevantes = train.loc[filtro, :]        # dataframe com tweets relevantes

filtro2 = train.Classificação==0        # filtra os tweets classificados como irrelevantes
irrelevantes = train.loc[filtro2, :]    # dataframe com tweets irrelevantes

In [49]:
# Transforma o dataframe relevantes em um único texto
string_relevante = ''
for linha in relevantes.Treinamento:
    string_relevante+= ' '+ linha
string_relevante

' twitter vai fazer compr airfryer quero assar pão queijo forno daqui casa bosta posso fazer airfryer limp antes enfim vou passar fome preguiça bom aparentemente deu ruim saiu toda tinta / teflon airfryer mano tô louca nessa airfryer serase boa frango cubos batata doce cenoura picados aquecidos airfryer selados frigideira molho cebola queijos cebola dourada mantega creme leite queijo minas padrão cream cheese philadelphia coisas superestimadas batata frita airfryer 🤡 realidade sei viver airfryer aniversário tá chegando alguém quiser dar presente tô aceitando airfryer manha compr airfryer polishop c cartao salnorabo ninguém quer dar presente airfryer tapete dadinho q vc compr mercado aí faço airfryer fica divino recomendo vamos conseguir estragar batata frita airfryer milésima vez 😛 lembrei airfryer agora todos problemas vida sumiram cama sofá novo chegaram aqui apto final semana chega aspirador pó ✅ próxima aquisição airfryer abençoa senhor abençoa filha fatura cartão crédito próximos 

In [50]:
# Transforma o dataframe irrelevantes em um único texto
string_irrelevante = ''
for linha in irrelevantes.Treinamento:
    string_irrelevante+= ' '+ linha
string_irrelevante

' airfryer panelas polishop celebra anos descontos r ajudando país economizar energia assando torresmo airfryer caras sonhei airfryer donos airfryer seita toda certeza q aqui casa uso airfryer reina 😅 preocupada vídeo game pai cachorro mãe airfryer ser airfryer minutinhos airfryer tá croc croc novo dá secada óleo tbm pessoal vem hospedar hotel agora airfryer trazem pro rolê mano tá farofada sdds tempo tibau sul pipa destinos internacionais vivendo semana fogão explorando todas possibilidades culinárias airfryer coisas cujos admiradores / participantes organizam tal seita ciro gomes airfryer bolsonaro astrologia fone chinês culpa lula bdsm juliette kpop pirâmide financeira criptomoedas ciclismo apple xiaomi continuem replys jantarzinho especial q vou preparar hj = tacar pao alho airfryer arrependida ter concordado mãe q inútil trazer airfryer pra bt 😫 taynah assistido jogos fluminense roger airfryer uns insrantes fiquei pensando diabos alguém queimou airfryer haha pastel airfryer ficou 

In [51]:
todas_relevantes = string_relevante.split()        #Lista com todas as palavras que aparece na string relevante
todas_irrelevantes = string_irrelevante.split()    #Lista com todas as palavras que aparece na string irrelevante

serie_relevante = pd.Series(todas_relevantes)      #Transforma a lista de strings relevantes em um series
serie_irrelevante = pd.Series(todas_irrelevantes)  #Transforma a lista de strings irrelevantes em um series

In [52]:
tabela_relevante_relativa = serie_relevante.value_counts(True)
tabela_relevante_relativa                                        # percentual de cada palavra da series relevante

airfryer     0.098015
compr        0.015509
pra          0.013027
batata       0.012407
q            0.012407
               ...   
crianças     0.000620
assadinho    0.000620
🍨            0.000620
porções      0.000620
tempera      0.000620
Length: 826, dtype: float64

In [53]:
tabela_irrelevante_relativa = serie_irrelevante.value_counts(True)
tabela_irrelevante_relativa.head(50)                                        # percentual de cada palavra da series irrelevante

airfryer      0.095358
pra           0.011920
q             0.009410
compr         0.006274
kk            0.006274
fazer         0.006274
n             0.006274
casa          0.006274
batata        0.005646
aqui          0.005646
usar          0.005646
óleo          0.005646
tô            0.005019
hoje          0.005019
vai           0.004391
pode          0.004391
pão           0.004391
mundo         0.004391
energia       0.003764
dentro        0.003764
ser           0.003764
quente        0.003764
queijo        0.003764
fica          0.003764
alho          0.003764
ficou         0.003764
vida          0.003764
tava          0.003764
ter           0.003764
mãe           0.003764
deus          0.003764
melhor        0.003764
gás           0.003764
fogão         0.003137
luz           0.003137
ventilador    0.003137
assim         0.003137
tudo          0.003137
sempre        0.003137
microondas    0.003137
ar            0.003137
coisas        0.003137
bom           0.003137
tempo      

In [54]:
todas = string_irrelevante + string_relevante          # juntando todas as palavras que aparecem na base de dados Treinamento
todas_as_palavras = todas.split()                      # lista com todas as palavras
serie_total = pd.Series(todas_as_palavras) 
serie_total
# tranforma a lista em Series
# tabela_total_relativa = serie_total.value_counts(True)
# tabela_total_relativa                                   # percentual de cada palavra em relação a todas as palavras

0       airfryer
1        panelas
2       polishop
3        celebra
4           anos
          ...   
3201       ranço
3202      batata
3203      palito
3204       feita
3205    airfryer
Length: 3206, dtype: object

In [55]:
P_R = len(todas_relevantes)/len(todas_as_palavras)        # probabilidade de uma palavra ser relevante
P_I = len(todas_irrelevantes)/len(todas_as_palavras)      # probabilidade de uma palavra ser irrelevante
P_I + P_R                                                 # soma das probabilidade para verificar se está correto

1.0

In [56]:
freq_rel_absoluta = serie_relevante.value_counts()           # quantidade de cada palavra na serie relevante
freq_irrel_absoluta = serie_irrelevante.value_counts()       # quantidade de cada palavra na serie irrelevante
freq_tot_absoluta = serie_total.value_counts()               # quantidade de cada palavra na serie total

# Função Classificadora Naive-Bayes

In [57]:
def classificador(dataframe, series):                         #recebe o dataframe e a serie de tweets que será classificada
    
    classificados = []
    for frase in series:
        lista_palavras = frase.split()
        lista_palavras = retirar_stopwords(lista_palavras)    #implementa função que tira as stopwords
        k,i = 0,0                                             #contador de palavras NÃO presentes em relevantes (k) e em irrevelantes (i)
        for palavra in lista_palavras:
            if palavra not in todas_relevantes: 
                k += 1
            if palavra not in todas_irrelevantes:
                i += 1

        prob_frase_dado_relevante = 1
        prob_frase_dado_irrelevante = 1

        if k > 0:                                   #se tiver palavras no series que não estao na nossa base de dados relevantes
            for palavra in lista_palavras:
                if palavra in freq_rel_absoluta:
                    prob = (freq_rel_absoluta[palavra]+ 1 ) / (freq_rel_absoluta.sum()+ len(freq_tot_absoluta)) # SUAVIZAÇÃO DE LAPLACE"
                else: 
                    prob = (0 + 1 ) / (freq_rel_absoluta.sum()+ len(freq_tot_absoluta))
                prob_frase_dado_relevante *= prob           #multiplica a probabilidade de cada palavra dado frase
        else:
            for palavra in lista_palavras:
                prob = freq_rel_absoluta[palavra] / freq_rel_absoluta.sum()
                prob_frase_dado_relevante *= prob           #multiplica a probabilidade de cada palavra dado frase

        if i > 0:
            for palavra in lista_palavras:
                if palavra in freq_irrel_absoluta:
                    prob = ( freq_irrel_absoluta[palavra]+ 1 ) / (freq_irrel_absoluta.sum() +len(freq_tot_absoluta))
                else:
                    prob = ( 0 + 1 ) / (freq_irrel_absoluta.sum() +len(freq_tot_absoluta))
                prob_frase_dado_irrelevante *= prob         #multiplica a probabilidade de cada palavra dado frase
        else:
            for palavra in lista_palavras:
                prob = ( freq_irrel_absoluta[palavra] ) / (freq_irrel_absoluta.sum())
                prob_frase_dado_irrelevante *= prob         #multiplica a probabilidade de cada palavra dado frase
                
        probRdadoFrase = prob_frase_dado_relevante* P_R
        probIdadoFrase = prob_frase_dado_irrelevante* P_I
        
        if probRdadoFrase > probIdadoFrase:
            classificados.append(1)                         #classifica como relevante
        else:  
            classificados.append(0)                         #classifica como irrelevante
            
    dataframe['Classificado'] = pd.Series(classificados)    #adiciona uma nova coluna ao dataframe com a classificação
    return dataframe
                

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [58]:
test

,Teste,Classificação
0,"o bom da airfryer fica tudo pronto rápido, não...",1
1,um app pra calcular se é mais barato fazer a c...,0
2,@raytotsukishiro aumentou o preço de tudo. aqu...,0
3,meti um pão de queijo na airfryer 😌,1
4,"@alcysio @safbf pô alcysio, me adaptei total p...",0
...,...,...
195,tutorial bolinha de queijo✨\n\n200g de queijo ...,1
196,pessoal daqui de casa inventou de querer fazer...,1
197,"eu nao sei voces, mas eu ganhei uma airfryer g...",1
198,@ibebelly amanhã eu vou fazer um pão de alho d...,1


In [59]:
lista = []

for linha in test['Teste']:
    limpo = cleanup(linha.lower())             # Limpa a planilha Teste e tira as letras maiúsculas
    lista.append(limpo)

test['Teste'] = pd.Series(lista)             # Substitui os tweets originais da planilha, pelos tweets limpos
test

,Teste,Classificação
0,o bom da airfryer fica tudo pronto rápido não ...,1
1,um app pra calcular se é mais barato fazer a c...,0
2,aumentou o preço de tudo aqui a gente compr um...,0
3,meti um pão de queijo na airfryer 😌,1
4,pô alcysio me adaptei total pra usar o mínimo ...,0
...,...,...
195,tutorial bolinha de queijo ✨ g de queijo muçar...,1
196,pessoal daqui de casa inventou de querer fazer...,1
197,eu nao sei voces mas eu ganhei uma airfryer gi...,1
198,amanhã eu vou fazer um pão de alho de airfryer...,1


# Implementando a função...

In [60]:
df = test               # dataframe que será argumento da função classificadora
series= test['Teste']   # series que será argumento da função classificadora

In [61]:
# Implementa função
df_classificado = classificador(df, series)
df_classificado

,Teste,Classificação,Classificado
0,o bom da airfryer fica tudo pronto rápido não ...,1,1
1,um app pra calcular se é mais barato fazer a c...,0,1
2,aumentou o preço de tudo aqui a gente compr um...,0,1
3,meti um pão de queijo na airfryer 😌,1,0
4,pô alcysio me adaptei total pra usar o mínimo ...,0,1
...,...,...,...
195,tutorial bolinha de queijo ✨ g de queijo muçar...,1,0
196,pessoal daqui de casa inventou de querer fazer...,1,1
197,eu nao sei voces mas eu ganhei uma airfryer gi...,1,1
198,amanhã eu vou fazer um pão de alho de airfryer...,1,1


In [62]:
filtro_1 = df_classificado.Classificação == 1
nova = df_classificado.loc[filtro_1]
nova    #planilha que classificamos como relevante

,Teste,Classificação,Classificado
0,o bom da airfryer fica tudo pronto rápido não ...,1,1
3,meti um pão de queijo na airfryer 😌,1,0
5,e vamos de tentar fritar batata na airfryer,1,0
11,eu tal hora pesquisando se dar pra fazer paste...,1,1
12,aqui na casa da minha mãe geralmente se usa de...,1,0
...,...,...,...
195,tutorial bolinha de queijo ✨ g de queijo muçar...,1,0
196,pessoal daqui de casa inventou de querer fazer...,1,1
197,eu nao sei voces mas eu ganhei uma airfryer gi...,1,1
198,amanhã eu vou fazer um pão de alho de airfryer...,1,1


In [63]:
filtro1_new = df_classificado.Classificado == 1
new = df_classificado.loc[filtro1_new]
new    #planilha que foi classificada pela função como relevante

,Teste,Classificação,Classificado
0,o bom da airfryer fica tudo pronto rápido não ...,1,1
1,um app pra calcular se é mais barato fazer a c...,0,1
2,aumentou o preço de tudo aqui a gente compr um...,0,1
4,pô alcysio me adaptei total pra usar o mínimo ...,0,1
8,e agora que não sei se uso mais a airfryer ou ...,0,1
...,...,...,...
194,quem não tem uma invista melhor coisa da vida ...,1,1
196,pessoal daqui de casa inventou de querer fazer...,1,1
197,eu nao sei voces mas eu ganhei uma airfryer gi...,1,1
198,amanhã eu vou fazer um pão de alho de airfryer...,1,1


## Extraindo as contagens para verificação de performance

In [70]:
tabela = pd.crosstab(df_classificado.Classificado, df_classificado.Classificação, normalize= True)  # tabela de porcentagens
tabela

Classificação,0,1
Classificado,,
0,0.215,0.155
1,0.235,0.395


Porcentagem de verdadeiros positivos (mensagens relevantes e que são
classificadas como relevantes)

In [71]:
verdadeiros_positivos = tabela.iloc[1,1]*100
verdadeiros_positivos

39.5

Porcentagem de falsos positivos (mensagens irrelevantes e que são classificadas
como relevantes)

In [72]:
falsos_positivos = tabela.iloc[1,0]*100
falsos_positivos

23.5

Porcentagem de verdadeiros negativos (mensagens irrelevantes e que são
classificadas como irrelevantes)

In [73]:
verdadeiros_negativos = tabela.iloc[0,0]*100
verdadeiros_negativos

21.5

Porcentagem de falsos negativos (mensagens relevantes e que são classificadas
como irrelevantes)

In [74]:
falsos_negativos = tabela.iloc[0,1]*100
falsos_negativos

15.5

Acurácia (mensagens corretamente classificadas, independente da categoria)

In [75]:
eficacia = verdadeiros_positivos + verdadeiros_negativos
eficacia

61.0

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**